In [1]:
import pandas as pd

## Overview of Met Police and City of London Police datasets (on street crime)

In [2]:
met_df = pd.read_csv('../data/2018-07-metropolitan-street.csv')

print(met_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95677 entries, 0 to 95676
Data columns (total 12 columns):
Crime ID                 74480 non-null object
Month                    95677 non-null object
Reported by              95677 non-null object
Falls within             95677 non-null object
Longitude                94577 non-null float64
Latitude                 94577 non-null float64
Location                 95677 non-null object
LSOA code                94577 non-null object
LSOA name                94577 non-null object
Crime type               95677 non-null object
Last outcome category    74480 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 8.8+ MB
None


In [3]:
city_df = pd.read_csv('../data/2018-07-city-of-london-street.csv')

print(city_df.info())

city_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 12 columns):
Crime ID                 627 non-null object
Month                    859 non-null object
Reported by              859 non-null object
Falls within             859 non-null object
Longitude                778 non-null float64
Latitude                 778 non-null float64
Location                 859 non-null object
LSOA code                778 non-null object
LSOA name                778 non-null object
Crime type               859 non-null object
Last outcome category    627 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 80.6+ KB
None


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,bf66549a084866d616a3f972ad9ad2bbbda6651ca48266...,2018-07,City of London Police,City of London Police,-0.119224,51.529042,On or near Hospital,E01000939,Camden 024C,Other theft,Investigation complete; no suspect identified,NaN
1,NaN,2018-07,City of London Police,City of London Police,-0.106453,51.518207,On or near Charterhouse Street,E01000916,Camden 027B,Anti-social behaviour,NaN,NaN
2,NaN,2018-07,City of London Police,City of London Police,-0.111497,51.518226,On or near Pedestrian Subway,E01000914,Camden 028B,Anti-social behaviour,NaN,NaN
3,NaN,2018-07,City of London Police,City of London Police,-0.111497,51.518226,On or near Pedestrian Subway,E01000914,Camden 028B,Anti-social behaviour,NaN,NaN
4,NaN,2018-07,City of London Police,City of London Police,-0.111497,51.518226,On or near Pedestrian Subway,E01000914,Camden 028B,Anti-social behaviour,NaN,NaN


In [4]:
london_df = pd.concat([met_df, city_df], ignore_index=True).dropna(subset=['Longitude', 'Latitude'])

In [5]:
london_df['Coordinate'] = london_df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

In [6]:
crime_freq = london_df.groupby('Coordinate').size()
crime_freq.count()

33513

In [7]:
crime_freq.head()

Coordinate
(-5.482662, 50.211896)              2
(-3.3954779999999998, 51.951301)    1
(-3.191991, 51.46346)               1
(-3.016167, 53.3255)                1
(-2.9570790000000002, 53.473227)    1
dtype: int64

In [8]:
print(crime_freq.min())
print(crime_freq.max())

1
239


In [9]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    
    for index, value in crime_freq.iteritems():
        data = {
            'type': 'Feature',
            'properties': {
                'crimes': value
            },
            'geometry': {
                'type': 'Point',
                'coordinates': [index[0], index[1]]
            }
        }
        
        template['features'].append(data)

with open('../london_july_2018.geojson', 'w') as output:
    json.dump(template, output)

In [10]:
print(len(template['features']))

33513


In [11]:
template['features'][0]

{'geometry': {'coordinates': [-5.482662, 50.211896], 'type': 'Point'},
 'properties': {'crimes': 2},
 'type': 'Feature'}